In [41]:
import numpy as np
import pandas as pd
import random
import pickle
import itertools
import folium #mapping
import seaborn as sns #color palette
#from colormap import rgb2hex #chnage RGB codes to hex codes

In [2]:
## USE THIS CODE TO IMPORT DATA

file_name = 'data/results/results_toydata.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

##[(supplier_lat, supplier_lon), (receiver_lat, receiver_lon), distance, volume_transferred]
#loaded_list[0:5]

In [134]:
#convert to dataframe

df = pd.DataFrame(loaded_list, columns = ['supplier', 'receiver', 'distance', 'volume'])
df.head()
#df.shape #109031, 4

#list-ify the suppliers to get rid of weird numpy object datatype
supplier_split = df.supplier.to_list()
receiver_split = df.receiver.to_list()
#supplier_split[0:5]


#convert suppliers to lat and lon
supplier_lat= [supplier_split[j][0] for j in range(0,len(supplier_split))]
supplier_lon = [supplier_split[j][1] for j in range(0,len(supplier_split))]

#convert to list of receiver lat and long
receiver_lat= [receiver_split[j][0] for j in range(0,len(receiver_split))]
receiver_lon = [receiver_split[j][1] for j in range(0,len(receiver_split))]

#add back into the df
df['supplier_lat'] = supplier_lat
df['supplier_lon'] = supplier_lon
df['receiver_lat'] = receiver_lat
df['receiver_lon'] = receiver_lat

df.head()

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[36.23867035, -120.38221741]",49.705671,363.79758,36.92375,-120.10412,36.238670,36.238670
1,"[36.92375, -120.10412]","[36.24571609, -120.33669281]",48.508588,955.40343,36.92375,-120.10412,36.245716,36.245716
2,"[36.92375, -120.10412]","[36.25873947, -120.32772064]",47.510457,818.13738,36.92375,-120.10412,36.258739,36.258739
3,"[36.92375, -120.10412]","[36.224933, -120.303681]",49.447437,240.09383,36.92375,-120.10412,36.224933,36.224933
4,"[36.92375, -120.10412]","[36.255945, -120.34494]",47.954999,158.00688,36.92375,-120.10412,36.255945,36.255945


In [135]:
#subset data for initial map
df_small = df.head(1000)

In [137]:

#create a color paltte
palette = sns.color_palette(None, 10)
palette

palette[0]

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765)

In [136]:
df_small

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[36.23867035, -120.38221741]",49.705671,363.79758,36.92375,-120.10412,36.238670,36.238670
1,"[36.92375, -120.10412]","[36.24571609, -120.33669281]",48.508588,955.40343,36.92375,-120.10412,36.245716,36.245716
2,"[36.92375, -120.10412]","[36.25873947, -120.32772064]",47.510457,818.13738,36.92375,-120.10412,36.258739,36.258739
3,"[36.92375, -120.10412]","[36.224933, -120.303681]",49.447437,240.09383,36.92375,-120.10412,36.224933,36.224933
4,"[36.92375, -120.10412]","[36.255945, -120.34494]",47.954999,158.00688,36.92375,-120.10412,36.255945,36.255945
...,...,...,...,...,...,...,...,...
995,"[37.4166, -122.03599]","[35.26726913, -119.34642029]",210.863250,560.24732,37.41660,-122.03599,35.267269,35.267269
996,"[37.4166, -122.03599]","[35.29407501, -119.52223969]",202.649803,325.88290,37.41660,-122.03599,35.294075,35.294075
997,"[37.4166, -122.03599]","[35.2630043, -119.45105743]",206.966271,306.94616,37.41660,-122.03599,35.263004,35.263004
998,"[37.4166, -122.03599]","[35.2785759, -119.43296814]",206.894146,766.79124,37.41660,-122.03599,35.278576,35.278576


In [139]:
df.shape

(109031, 8)

In [142]:
df['supplier_lat'][1000]

37.4166

In [152]:
#this_lat = df_small['supplier_lat'][0]
#this_lon = df_small['supplier_lon'][0]

In [156]:
#pseudocode
#find unique suppliers (use original column, then split)
#cycle through the colors
#for each color plot all receivers
#plot the supplirs as black or something for now

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[36.23867035, -120.38221741]",49.705671,363.79758,36.92375,-120.10412,36.238670,36.238670
1,"[36.92375, -120.10412]","[36.24571609, -120.33669281]",48.508588,955.40343,36.92375,-120.10412,36.245716,36.245716
2,"[36.92375, -120.10412]","[36.25873947, -120.32772064]",47.510457,818.13738,36.92375,-120.10412,36.258739,36.258739
3,"[36.92375, -120.10412]","[36.224933, -120.303681]",49.447437,240.09383,36.92375,-120.10412,36.224933,36.224933
4,"[36.92375, -120.10412]","[36.255945, -120.34494]",47.954999,158.00688,36.92375,-120.10412,36.255945,36.255945
...,...,...,...,...,...,...,...,...
110,"[36.92375, -120.10412]","[35.26367569, -119.57122803]",118.278557,475.81142,36.92375,-120.10412,35.263676,35.263676
111,"[36.92375, -120.10412]","[35.08056641, -119.36569977]",133.644013,318.74636,36.92375,-120.10412,35.080566,35.080566
112,"[36.92375, -120.10412]","[34.03170013, -118.07074738]",229.985439,911.59735,36.92375,-120.10412,34.031700,34.031700
113,"[36.92375, -120.10412]","[35.43372726, -119.13058472]",116.256245,847.48562,36.92375,-120.10412,35.433727,35.433727


In [159]:
#create map
#mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=6 ) #initialize map location and zoom
suppliers = []

#loop through suppliers, paint all the receivers in the same color
for i in range(0,df_small.shape[0]-1):
    #suppliers.append(df_small['supplier_lat'][i], df_small['supplier_lon'][i])) #keep track of covered suppliers; don't re-map them
    this_lat = df_small['supplier_lat'][i]
    this_lon = df_small['supplier_lon'][i]
    receivers_1 = df_small[df_small['supplier_lat'] == this_lat]
    receivers_2 = receivers_1[receivers_1['supplier_lon'] == this_lon]
    print(receivers_2)
    
    #for j in receivers:
     #   folium.Marker( location=[ j ], fill_color=palette[i], radius=8 ).add_to( mapit )
    


                   supplier                      receiver    distance  \
0    [36.92375, -120.10412]  [36.23867035, -120.38221741]   49.705671   
1    [36.92375, -120.10412]  [36.24571609, -120.33669281]   48.508588   
2    [36.92375, -120.10412]  [36.25873947, -120.32772064]   47.510457   
3    [36.92375, -120.10412]      [36.224933, -120.303681]   49.447437   
4    [36.92375, -120.10412]       [36.255945, -120.34494]   47.954999   
..                      ...                           ...         ...   
110  [36.92375, -120.10412]  [35.26367569, -119.57122803]  118.278557   
111  [36.92375, -120.10412]  [35.08056641, -119.36569977]  133.644013   
112  [36.92375, -120.10412]  [34.03170013, -118.07074738]  229.985439   
113  [36.92375, -120.10412]  [35.43372726, -119.13058472]  116.256245   
114  [36.92375, -120.10412]  [35.50600052, -118.90733337]  118.432413   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
0    363.79758      36.92375    -120.10412     36.

                   supplier                      receiver    distance  \
0    [36.92375, -120.10412]  [36.23867035, -120.38221741]   49.705671   
1    [36.92375, -120.10412]  [36.24571609, -120.33669281]   48.508588   
2    [36.92375, -120.10412]  [36.25873947, -120.32772064]   47.510457   
3    [36.92375, -120.10412]      [36.224933, -120.303681]   49.447437   
4    [36.92375, -120.10412]       [36.255945, -120.34494]   47.954999   
..                      ...                           ...         ...   
110  [36.92375, -120.10412]  [35.26367569, -119.57122803]  118.278557   
111  [36.92375, -120.10412]  [35.08056641, -119.36569977]  133.644013   
112  [36.92375, -120.10412]  [34.03170013, -118.07074738]  229.985439   
113  [36.92375, -120.10412]  [35.43372726, -119.13058472]  116.256245   
114  [36.92375, -120.10412]  [35.50600052, -118.90733337]  118.432413   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
0    363.79758      36.92375    -120.10412     36.

                   supplier                      receiver    distance  \
0    [36.92375, -120.10412]  [36.23867035, -120.38221741]   49.705671   
1    [36.92375, -120.10412]  [36.24571609, -120.33669281]   48.508588   
2    [36.92375, -120.10412]  [36.25873947, -120.32772064]   47.510457   
3    [36.92375, -120.10412]      [36.224933, -120.303681]   49.447437   
4    [36.92375, -120.10412]       [36.255945, -120.34494]   47.954999   
..                      ...                           ...         ...   
110  [36.92375, -120.10412]  [35.26367569, -119.57122803]  118.278557   
111  [36.92375, -120.10412]  [35.08056641, -119.36569977]  133.644013   
112  [36.92375, -120.10412]  [34.03170013, -118.07074738]  229.985439   
113  [36.92375, -120.10412]  [35.43372726, -119.13058472]  116.256245   
114  [36.92375, -120.10412]  [35.50600052, -118.90733337]  118.432413   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
0    363.79758      36.92375    -120.10412     36.

                   supplier                      receiver    distance  \
0    [36.92375, -120.10412]  [36.23867035, -120.38221741]   49.705671   
1    [36.92375, -120.10412]  [36.24571609, -120.33669281]   48.508588   
2    [36.92375, -120.10412]  [36.25873947, -120.32772064]   47.510457   
3    [36.92375, -120.10412]      [36.224933, -120.303681]   49.447437   
4    [36.92375, -120.10412]       [36.255945, -120.34494]   47.954999   
..                      ...                           ...         ...   
110  [36.92375, -120.10412]  [35.26367569, -119.57122803]  118.278557   
111  [36.92375, -120.10412]  [35.08056641, -119.36569977]  133.644013   
112  [36.92375, -120.10412]  [34.03170013, -118.07074738]  229.985439   
113  [36.92375, -120.10412]  [35.43372726, -119.13058472]  116.256245   
114  [36.92375, -120.10412]  [35.50600052, -118.90733337]  118.432413   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
0    363.79758      36.92375    -120.10412     36.

                   supplier                      receiver    distance  \
0    [36.92375, -120.10412]  [36.23867035, -120.38221741]   49.705671   
1    [36.92375, -120.10412]  [36.24571609, -120.33669281]   48.508588   
2    [36.92375, -120.10412]  [36.25873947, -120.32772064]   47.510457   
3    [36.92375, -120.10412]      [36.224933, -120.303681]   49.447437   
4    [36.92375, -120.10412]       [36.255945, -120.34494]   47.954999   
..                      ...                           ...         ...   
110  [36.92375, -120.10412]  [35.26367569, -119.57122803]  118.278557   
111  [36.92375, -120.10412]  [35.08056641, -119.36569977]  133.644013   
112  [36.92375, -120.10412]  [34.03170013, -118.07074738]  229.985439   
113  [36.92375, -120.10412]  [35.43372726, -119.13058472]  116.256245   
114  [36.92375, -120.10412]  [35.50600052, -118.90733337]  118.432413   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
0    363.79758      36.92375    -120.10412     36.

...

                  supplier                      receiver    distance  \
346  [37.4166, -122.03599]  [33.92402267, -117.84706879]  336.840558   
347  [37.4166, -122.03599]   [33.92357635, -117.8470459]  336.863891   
348  [37.4166, -122.03599]  [33.92441177, -117.84135437]  337.045757   
349  [37.4166, -122.03599]   [33.92301941, -117.8447876]  336.980672   
350  [37.4166, -122.03599]   [33.92248917, -117.8445816]  337.015420   
..                     ...                           ...         ...   
995  [37.4166, -122.03599]  [35.26726913, -119.34642029]  210.863250   
996  [37.4166, -122.03599]  [35.29407501, -119.52223969]  202.649803   
997  [37.4166, -122.03599]   [35.2630043, -119.45105743]  206.966271   
998  [37.4166, -122.03599]   [35.2785759, -119.43296814]  206.894146   
999  [37.4166, -122.03599]  [35.25614548, -119.36200714]  210.794349   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
346  816.79495       37.4166    -122.03599     33.924023     3

                  supplier                      receiver    distance  \
346  [37.4166, -122.03599]  [33.92402267, -117.84706879]  336.840558   
347  [37.4166, -122.03599]   [33.92357635, -117.8470459]  336.863891   
348  [37.4166, -122.03599]  [33.92441177, -117.84135437]  337.045757   
349  [37.4166, -122.03599]   [33.92301941, -117.8447876]  336.980672   
350  [37.4166, -122.03599]   [33.92248917, -117.8445816]  337.015420   
..                     ...                           ...         ...   
995  [37.4166, -122.03599]  [35.26726913, -119.34642029]  210.863250   
996  [37.4166, -122.03599]  [35.29407501, -119.52223969]  202.649803   
997  [37.4166, -122.03599]   [35.2630043, -119.45105743]  206.966271   
998  [37.4166, -122.03599]   [35.2785759, -119.43296814]  206.894146   
999  [37.4166, -122.03599]  [35.25614548, -119.36200714]  210.794349   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
346  816.79495       37.4166    -122.03599     33.924023     3

                  supplier                      receiver    distance  \
346  [37.4166, -122.03599]  [33.92402267, -117.84706879]  336.840558   
347  [37.4166, -122.03599]   [33.92357635, -117.8470459]  336.863891   
348  [37.4166, -122.03599]  [33.92441177, -117.84135437]  337.045757   
349  [37.4166, -122.03599]   [33.92301941, -117.8447876]  336.980672   
350  [37.4166, -122.03599]   [33.92248917, -117.8445816]  337.015420   
..                     ...                           ...         ...   
995  [37.4166, -122.03599]  [35.26726913, -119.34642029]  210.863250   
996  [37.4166, -122.03599]  [35.29407501, -119.52223969]  202.649803   
997  [37.4166, -122.03599]   [35.2630043, -119.45105743]  206.966271   
998  [37.4166, -122.03599]   [35.2785759, -119.43296814]  206.894146   
999  [37.4166, -122.03599]  [35.25614548, -119.36200714]  210.794349   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
346  816.79495       37.4166    -122.03599     33.924023     3

                  supplier                      receiver    distance  \
346  [37.4166, -122.03599]  [33.92402267, -117.84706879]  336.840558   
347  [37.4166, -122.03599]   [33.92357635, -117.8470459]  336.863891   
348  [37.4166, -122.03599]  [33.92441177, -117.84135437]  337.045757   
349  [37.4166, -122.03599]   [33.92301941, -117.8447876]  336.980672   
350  [37.4166, -122.03599]   [33.92248917, -117.8445816]  337.015420   
..                     ...                           ...         ...   
995  [37.4166, -122.03599]  [35.26726913, -119.34642029]  210.863250   
996  [37.4166, -122.03599]  [35.29407501, -119.52223969]  202.649803   
997  [37.4166, -122.03599]   [35.2630043, -119.45105743]  206.966271   
998  [37.4166, -122.03599]   [35.2785759, -119.43296814]  206.894146   
999  [37.4166, -122.03599]  [35.25614548, -119.36200714]  210.794349   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
346  816.79495       37.4166    -122.03599     33.924023     3

                  supplier                      receiver    distance  \
346  [37.4166, -122.03599]  [33.92402267, -117.84706879]  336.840558   
347  [37.4166, -122.03599]   [33.92357635, -117.8470459]  336.863891   
348  [37.4166, -122.03599]  [33.92441177, -117.84135437]  337.045757   
349  [37.4166, -122.03599]   [33.92301941, -117.8447876]  336.980672   
350  [37.4166, -122.03599]   [33.92248917, -117.8445816]  337.015420   
..                     ...                           ...         ...   
995  [37.4166, -122.03599]  [35.26726913, -119.34642029]  210.863250   
996  [37.4166, -122.03599]  [35.29407501, -119.52223969]  202.649803   
997  [37.4166, -122.03599]   [35.2630043, -119.45105743]  206.966271   
998  [37.4166, -122.03599]   [35.2785759, -119.43296814]  206.894146   
999  [37.4166, -122.03599]  [35.25614548, -119.36200714]  210.794349   

        volume  supplier_lat  supplier_lon  receiver_lat  receiver_lon  
346  816.79495       37.4166    -122.03599     33.924023     3

In [ ]:
# #round two, with clusters

# mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=6 )


# mc = MarkerCluster()
# # add markers to map

# for supplier in unique_suppliers:
#  mc.add_child(folium.Marker(location=supplier, 
#  popup= (“City: {city}<br>”
#  “Population: {pop}<br>”
#  “BreweryCount: {count}<br>”
#  “People per Brewery: {ppb}”
#  ).format(city=row.city, 
#  pop=row.population, 
#  count=row.number_of_breweries, 
#  ppb=row.people_per_brewery)))
 
# mapit.add_child(mc)
# mapit.save(‘WestCoastBreweries.html’)
# mapit

In [16]:
# #latlon = records[0:1000] #[ (51.249443914705175, -0.13878830247011467), (51.249443914705175, -0.13878830247011467), (51.249768239976866, -2.8610415615063034)]
# latlon = df_small['receiver']
# mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=6 )
# for coord in latlon:
#     folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to( mapit )
    
# mapit